# Ajustando a tabela de ideias

In [ ]:
import pandas as pd

In [ ]:
file_path = 'Ideias.xlsx'

# Leitura da planilha Excel
df = pd.read_excel(file_path)

# Verificação inicial dos dados
df.head()

,IDEIA_TITULO,IDEIA_DESCRICAO,USUARIO,DATA_CRIACAO,COMPLEMENTO,RESPOSTA
0,Implementar chatbots que utilizem processament...,Em um cenário empresarial cada vez mais dinâmi...,jorge.souza,2023-11-17 15:55:54.000,Descreva melhor o desafio,O desafio principal na implementação de chatb...
1,NaN,NaN,NaN,NaN,Como a gente pode saber que deu certo a implem...,"Por exemplo, no CDIPE: Quantidade de currí..."
2,NaN,NaN,NaN,NaN,Quais o prazo das principais atividades?,NaN
3,NaN,NaN,NaN,NaN,Quais os principais custos(R$) dessa implement...,NaN
4,NaN,NaN,NaN,NaN,"Baseado nas entregas, monte uma apresentação p...",NaN


In [ ]:
# Manter apenas as colunas 'IDEIA_TITULO' e 'IDEIA_DESCRICAO'
df = df[['IDEIA_TITULO', 'IDEIA_DESCRICAO']]

# Remover todas as linhas onde qualquer valor em 'IDEIA_TITULO' ou 'IDEIA_DESCRICAO' é vazio
df = df.dropna(how='any')

# Criar a nova coluna 'IDEIA' que concatena 'IDEIA_TITULO' e 'IDEIA_DESCRICAO'
df['IDEIA'] = df['IDEIA_TITULO'].fillna('') + ' - ' + df['IDEIA_DESCRICAO'].fillna('')

# Reindexar o DataFrame
df = df.reset_index(drop=True)

# Exibir o DataFrame resultante
df.head()

,IDEIA_TITULO,IDEIA_DESCRICAO,IDEIA
0,Implementar chatbots que utilizem processament...,Em um cenário empresarial cada vez mais dinâmi...,Implementar chatbots que utilizem processament...
1,SN Conhecimento Compartilhado,Um espaço onde a sabedoria do Grupo SN é compa...,SN Conhecimento Compartilhado - Um espaço onde...
2,Subestação elétrica - Vitória Minerais,A unidade da Vitória Minerais está em grande e...,Subestação elétrica - Vitória Minerais - A uni...
3,Pintura e sinalizações na Fábrica,"Pintura de faixas de pedestre, caminho seguro ...",Pintura e sinalizações na Fábrica - Pintura d...
4,Contrução de um local cercado e limpo destinad...,Criar um espaço cercado e com chão limpo para...,Contrução de um local cercado e limpo destinad...


In [ ]:
# Criação do DataFrame df_ideias com as colunas especificadas, mantendo IDEIA_TITULO e IDEIA do DataFrame original
df_ideias = df[['IDEIA_TITULO', 'IDEIA']].copy()

# Adiciona as novas colunas com valores vazios
df_ideias = df_ideias.assign(
    categoria=pd.NA,
    impacto=pd.NA,
    reach=pd.NA,
    impact=pd.NA,
    confidence=pd.NA,
    business_value=pd.NA,
    time_criticality=pd.NA,
    risk_reduction=pd.NA
)

In [ ]:
df_ideias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   IDEIA_TITULO      135 non-null    object
 1   IDEIA             135 non-null    object
 2   categoria         0 non-null      object
 3   impacto           0 non-null      object
 4   reach             0 non-null      object
 5   impact            0 non-null      object
 6   confidence        0 non-null      object
 7   business_value    0 non-null      object
 8   time_criticality  0 non-null      object
 9   risk_reduction    0 non-null      object
dtypes: object(10)
memory usage: 10.7+ KB


# Langchain na veia - Priorização parte 1 (avaliação das ideias)

In [ ]:
import os
import openai
from langchain_openai import ChatOpenAI
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
# Configuração da API
openai.api_key = 'insira_sua_chave_aqui'
os.environ['OPENAI_API_KEY'] = openai.api_key
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [ ]:
# Definição dos esquemas de resposta
reach = ResponseSchema(name="reach", description="de 0.0 a 5.0; 5 significa maior alcance: quantas pessoas serão impactadas por essa iniciativa? (Considere para um mesmo período)")
impact = ResponseSchema(name="impact", description="de 0.0 a 1.0; 1 significa maior impacto: quanto isso irá impactar cada uma dessas pessoas?")
confidence = ResponseSchema(name="confidence", description="de 0.0 a 1.0; 1 significa 100% confiante: o quão confiante você está em relação a esses números?")
business_value = ResponseSchema(name="business_value", description="de 0.0 a 5.0; 5 significa maior valor: Qual a penalidade potencial ou impacto negativo se atrasar ou demorar? Quanto está deixando de ganhar com essa iniciativa?")
time_criticality = ResponseSchema(name="time_criticality", description="de 0.0 a 5.0; 5 significa extremamente crítico: O quão rápido o valor do negócio diminui ao longo do tempo? Os usuários irão esperar por nós ou encontrarão uma outra opção?")
risk_reduction = ResponseSchema(name="risk_reduction", description="de 0.0 a 5.0; 5 significa extremamente necessário para reduzir risco: Qual o risco de postergar esta iniciativa para o nosso negócio? Esta iniciativa vai abrir/facilitar novas oportunidades de negócio?")

response_schemas = [reach, impact, confidence, business_value, time_criticality, risk_reduction]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [ ]:
template = '''Você é um grande especialista em categorização de ideias, você trabalha em um grupo de empresas\
que atua no setor de mineração (brita e calcário) e construção civil (concreteira, construtora e incorporadora).\
Tem um total aproximado de 500 colaboradores, e tem como missão estrategica o crescimento sustentável do negocio.\

Nossos principais objetivos são:\
Calcário: Ampliar nossas operações agregando mais valor através da diversificação de receitas e da intimidade com nossos clientes.\
Concreto: Entregar o melhor concreto com a melhor experiência\
Britas: Ser referencia em produtividade, atendimento, sustentabilidade e qualidade dos produtos.\
Construtora: Transformar a experiencia de morar e investir atraves de produtos imobiliarios inovadores e experiencias incriveis\

para a seguinte ideia, Classifique-a, não precisa explicar nada use apenas os numeros na resposta\

reach: float - quantas pessoas serão impactadas por essa iniciativa?\
impact: float - quanto isso irá impactar cada uma dessas pessoas?\
confidence: float - o quão confiante você está em relação a esses números?\
business_value: float - Qual a penalidade potencial ou impacto negativo se atrasar ou demorar? Quanto está deixando de ganhar com essa iniciativa?\
time_criticality: float - O quão rápido o valor do negócio diminui ao longo do tempo? Os usuários irão esperar por nós ou encontrarão uma outra opção?\
risk_reduction: float - Qual o risco de postergar esta iniciativa para o nosso negócio? Esta iniciativa vai abrir/facilitar novas oportunidades de negócio?\
{format_instructions}

Ideia: {ideia}
'''

In [ ]:
prompt = ChatPromptTemplate.from_template(template=template)

In [ ]:
def classify_idea(idea):
    messages = prompt.format_messages(ideia=idea, format_instructions=format_instructions)
    response = llm(messages)
    output_dict = output_parser.parse(response.content)
    return output_dict

In [ ]:
for index, row in df_ideias.iterrows():
    idea = row['IDEIA']
    classification = classify_idea(idea)

    df_ideias.at[index, 'reach'] = classification.get('reach', '')
    df_ideias.at[index, 'impact'] = classification.get('impact', '')
    df_ideias.at[index, 'confidence'] = classification.get('confidence', '')
    df_ideias.at[index, 'business_value'] = classification.get('business_value', '')
    df_ideias.at[index, 'time_criticality'] = classification.get('time_criticality', '')
    df_ideias.at[index, 'risk_reduction'] = classification.get('risk_reduction', '')

C:\Users\decio.mendonca\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
print(df_ideias.head())

                                        IDEIA_TITULO  \
0  Implementar chatbots que utilizem processament...   
1                      SN Conhecimento Compartilhado   
2             Subestação elétrica - Vitória Minerais   
3                 Pintura e sinalizações na Fábrica    
4  Contrução de um local cercado e limpo destinad...   

                                               IDEIA categoria impacto reach  \
0  Implementar chatbots que utilizem processament...      <NA>    <NA>   4.0   
1  SN Conhecimento Compartilhado - Um espaço onde...      <NA>    <NA>   4.0   
2  Subestação elétrica - Vitória Minerais - A uni...      <NA>    <NA>   4.0   
3  Pintura e sinalizações na Fábrica  - Pintura d...      <NA>    <NA>   3.0   
4  Contrução de um local cercado e limpo destinad...      <NA>    <NA>   3.0   

  impact confidence business_value time_criticality risk_reduction  
0    0.8        0.9            4.0              4.0            5.0  
1    0.7        0.8            3.0          

# Categorização das ideias

In [ ]:
# Definição dos esquemas de resposta
categoria = ResponseSchema(name="categoria", description="Categoria da ideia: [Segurança e Saúde Ocupacional, Operações de Mineração, Construção e Infraestrutura, Sustentabilidade e Meio Ambiente, Tecnologia e Automação, Logística e Cadeia de Suprimentos, Gestão de Recursos Humanos, Relações com a Comunidade, Economia Circular, Energia e Eficiência Energética]")
impacto = ResponseSchema(name="impacto", description='''Grau de impacto para o negócio:
[Alto Impacto: Benefícios tangíveis: A ideia tem potencial para gerar benefícios significativos, como aumento de receita, redução de custos ou melhorias na eficiência operacional?, A ideia afeta várias áreas da organização ou tem potencial para beneficiar um grande número de pessoas ou processos?, A ideia representa uma abordagem inovadora que pode diferenciar a organização da concorrência ou estabelecer novos padrões?, A ideia contribui para a sustentabilidade da organização, seja ela ambiental, social ou financeira?, A ideia tem potencial para causar mudanças significativas na forma como a organização opera ou se posiciona no mercado? -  As ideias de alto impacto geralmente estão ligadas a criação de novos negocios ou novas fontes de receita significativas.;
Médio Impacto: A ideia contribui para a melhoria dos processos internos da organização, resultando em maior eficiência ou redução de custos?, A ideia pode melhorar a experiência dos stakeholders, como clientes, colaboradores ou fornecedores, aumentando a satisfação e a retenção?, A ideia ajuda a organização a se adaptar às mudanças no ambiente externo ou a tirar proveito de novas oportunidades de negócio?, A ideia promove a colaboração entre diferentes áreas da organização ou incentiva o compartilhamento de conhecimento e recursos?, A ideia pode contribuir para o desenvolvimento de competências dos colaboradores ou melhorar o ambiente de trabalho? - As ideias de medio impacto geralmente indicam a inserção de uma nova tecnologia, ou mudança significativa em processos e modelo de negocios;
Baixo Impacto: A ideia representa uma melhoria incremental em produtos, serviços ou processos existentes?, A ideia resolve um problema específico ou aborda uma questão pontual na organização?, A ideia pode ser implementada rapidamente e com poucos recursos ou investimentos?, A ideia pode melhorar a comunicação entre os colaboradores ou departamentos da organização?, A ideia pode resultar em pequenas economias de custos ou aumentar a eficiência em áreas específicas da organização? - Ideias de baixo impacto geralmente envolve pequenas melhorias, mas com um certo grau de inovação e perspectiva diferente para o negocio;
Ver e Agir: Ideias que são muito básicas, ou nem sequer se encaixam em um conceito de inovação, geralmente essas ideias são menos que incrementais e representam ações básicas que é só pegar e executar]''')

response_schemas = [categoria, impacto]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [ ]:
template = '''Você é um grande especialista em categorização de ideias, você trabalha em um grupo de empresas
que atua no setor de mineração (brita e calcário) e construção civil (concreteira, construtora e incorporadora).
Tem um total aproximado de 500 colaboradores, e tem como missão estrategica o crescimento sustentável do negocio.

Nossos principais objetivos são:
Calcário: Ampliar nossas operações agregando mais valor através da diversificação de receitas e da intimidade com nossos clientes.
Concreto: Entregar o melhor concreto com a melhor experiência
Britas: Ser referencia em produtividade, atendimento, sustentabilidade e qualidade dos produtos.
Construtora: Transformar a experiencia de morar e investir atraves de produtos imobiliarios inovadores e experiencias incriveis

Lembre-se de usar como referência o modelo Cynefin, onde ver e agir são ideias que resolvem problemas do espectro simples
Ideias de baixo impacto em geral envolvem problemas complicados
Ideias de médio impacto em geral resolvem problemas do espectro complexo
E por fim alto impacto são ideias relamente disruptivas e significativas.

Seja MUITO rigoroso e criterioso para classificar médio e alto impacto. Para baixo impacto seja razoavelmente rigoroso, todo resto é ver e agir.

categoria: String - Qual categoria a ideia se mostra mais aderente?\
impacto: String - Grau de impacto para o negócio: [Alto Impacto, Médio Impacto, Baixo Impacto, Ver e Agir]\
{format_instructions}

Ideia: {ideia}
'''

In [ ]:
prompt = ChatPromptTemplate.from_template(template=template)

In [ ]:
def classify_idea_category_impact(idea):
    messages = prompt.format_messages(ideia=idea, format_instructions=format_instructions)
    response = llm(messages)
    output_dict = output_parser.parse(response.content)
    return output_dict

In [ ]:
# Iterar sobre cada ideia e atualizar o DataFrame
for index, row in df_ideias.iterrows():
    idea = row['IDEIA']
    classification = classify_idea_category_impact(idea)

    df_ideias.at[index, 'categoria'] = classification.get('categoria', '')
    df_ideias.at[index, 'impacto'] = classification.get('impacto', '')


In [ ]:
df_ideias.head()

,IDEIA_TITULO,IDEIA,categoria,impacto,reach,impact,confidence,business_value,time_criticality,risk_reduction
0,Implementar chatbots que utilizem processament...,Implementar chatbots que utilizem processament...,Tecnologia e Automação,Médio Impacto,4.0,0.8,0.9,4.0,4.0,5.0
1,SN Conhecimento Compartilhado,SN Conhecimento Compartilhado - Um espaço onde...,Gestão de Recursos Humanos,Médio Impacto,4.0,0.7,0.8,3.0,3.0,4.0
2,Subestação elétrica - Vitória Minerais,Subestação elétrica - Vitória Minerais - A uni...,Energia e Eficiência Energética,Alto Impacto,4.0,0.8,0.9,4.5,4.0,5.0
3,Pintura e sinalizações na Fábrica,Pintura e sinalizações na Fábrica - Pintura d...,Segurança e Saúde Ocupacional,Baixo Impacto,3.0,0.7,0.8,4.0,3.0,4.0
4,Contrução de um local cercado e limpo destinad...,Contrução de um local cercado e limpo destinad...,Sustentabilidade e Meio Ambiente,Baixo Impacto,3.0,0.6,0.8,4.0,3.0,4.0


# Salvando Excel priorizado, categorizado e calculado

In [ ]:
# Função para converter colunas de str para float
def converter_para_float(df, colunas):
    for coluna in colunas:
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')  # Converte as colunas para float, erros são tratados como NaN

# Lista de colunas a serem convertidas
colunas_para_converter = ['reach', 'impact', 'confidence', 'business_value', 'time_criticality', 'risk_reduction']

# Converter as colunas para float
converter_para_float(df_ideias, colunas_para_converter)

# Função para calcular o valor de cada projeto
def calcular_valor(row):
    reach = row['reach']
    impact = row['impact']
    confidence = row['confidence']
    business_value = row['business_value']
    time_criticality = row['time_criticality']
    risk_reduction = row['risk_reduction']

    # Calcular o valor
    valor = (reach * impact * confidence) + ((business_value + time_criticality + risk_reduction) / 3)
    return valor

# Aplicação da função para calcular o valor
df_ideias['Valor'] = df_ideias.apply(calcular_valor, axis=1)

# Salvando os resultados em um novo arquivo Excel
df_ideias.to_excel('projetos_priorizados_langchain.xlsx', index=False)

# Comparar Ideias parecidas

In [ ]:
#Importar os tokenizers
#Transformar a tabela de ideias
#Usar como TXT
#Caçar ideias parecidas
#Estruturar em JSON, ou passar como parametro para busca manual
#Expor na tela

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

from transformers import GPT2TokenizerFast

import textract

In [ ]:
# Selecionar as colunas desejadas
df_comparar = df_ideias[['IDEIA_TITULO', 'IDEIA']]

# Salvar as informações no arquivo txt
with open('comparar_ideias.txt', 'w', encoding="utf-8") as file:
    for index, row in df_comparar.iterrows():
        titulo = row['IDEIA_TITULO']
        ideia = row['IDEIA']
        file.write(f"Título: {titulo}\nIdeia: {ideia}\n\n")

Tentando ler o documento

In [ ]:
with open('comparar_ideias.txt','r', encoding="utf-8") as f:
    text = f.read()

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

C:\Users\decio.mendonca\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key, model="text-embedding-ada-002")

db = FAISS.from_documents(chunks, embeddings)

C:\Users\decio.mendonca\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
nova_ideia = "Placas de identificação"

In [ ]:
query = f"Faça uma lista numerada com os titulos das ideias que tem relação com a seguinte ideia: {nova_ideia}"
docs = db.similarity_search(query)
docs[0]

Document(page_content='Título: Crachá de identificação para terceirizados \nIdeia: Crachá de identificação para terceirizados  - Um crachá de identificação para os funcionários que estão trabalhando terceirizados na fábrica, nele o prestador de serviços terá o seu nome, nome da empresa e o seu cargo. No verso os treinamentos que estão habilitados, treinamentos em altura, espaço confinado e etc.\n\nTítulo: CARRETO SN BRITA\nIdeia: CARRETO SN BRITA - Aquisição de caminhão e motorista para realizar entregas programadas/pré agendadas de brita. Isso minimiza filas transtornos de logística e tempo de espera para retirada do produto na planta de produção. Além disso agrega mais valor ao produto e pode se sustentar mediante cobrança de frete.\n\nTítulo: Energia para iluminação da Mina\nIdeia: Energia para iluminação da Mina - Atualmente a mina trabalha até às 22hs. Para iluminação é usado um pequeno gerador a gasolina e refletores. Nossa bomba da Mina utiliza uma tensão de 440v que atende e so

In [ ]:
chain = load_qa_chain(OpenAI(openai_api_key=openai.api_key, temperature=0), chain_type="stuff")

saida = chain.run(input_documents=docs, question=query)

In [ ]:
print(saida)


1. Crachá de identificação para terceirizados
2. CARRETO SN BRITA
3. Energia para iluminação da Mina
4. Criar um plano para limpeza em todos galpões para retirada da poluição visual
5. Desenhos e esquemas técnicos
6. Controle e melhoria nas operações com caminhões e equipamentos móveis
7. DEMONSTRATIVO DE CONTEÚDO MÁQUINA DE CAFÉ
8. PARCERIA PARA UTILIZAÇÃO DO AUDITÓRIO
9. Requisição de compra automática
10. Impressão das Apostilas de Inglês
11. Campeonato
12. Quadra de esportes


In [ ]:
# Remove espaços extras e quebra de linhas
saida = saida.strip()

# Divide a string em linhas
linhas = saida.split('\n')

# Cria um dicionário a partir das linhas
dicionario = {}
for linha in linhas:
    # Divide cada linha em número e item
    numero, item = linha.split('. ', 1)
    dicionario[int(numero)] = item

print(dicionario)

{1: 'Crachá de identificação para terceirizados', 2: 'CARRETO SN BRITA', 3: 'Energia para iluminação da Mina', 4: 'Criar um plano para limpeza em todos galpões para retirada da poluição visual', 5: 'Desenhos e esquemas técnicos', 6: 'Controle e melhoria nas operações com caminhões e equipamentos móveis', 7: 'DEMONSTRATIVO DE CONTEÚDO MÁQUINA DE CAFÉ', 8: 'PARCERIA PARA UTILIZAÇÃO DO AUDITÓRIO', 9: 'Requisição de compra automática', 10: 'Impressão das Apostilas de Inglês', 11: 'Campeonato', 12: 'Quadra de esportes'}


In [ ]:
# Filtra o DataFrame com base nos valores do dicionário
filtro_titulos = list(dicionario.values())
resultado = df_ideias[df_ideias['IDEIA_TITULO'].isin(filtro_titulos)]

resultado

,IDEIA_TITULO,IDEIA,categoria,impacto,reach,impact,confidence,business_value,time_criticality,risk_reduction,Valor
8,Desenhos e esquemas técnicos,Desenhos e esquemas técnicos - Em nossas unida...,Operações de Mineração,Médio Impacto,4.0,0.7,0.8,4.0,3.0,4.0,5.906667
39,DEMONSTRATIVO DE CONTEÚDO MÁQUINA DE CAFÉ,DEMONSTRATIVO DE CONTEÚDO MÁQUINA DE CAFÉ - Co...,Gestão de Recursos Humanos,Ver e Agir,2.0,0.3,0.8,1.0,1.0,1.0,1.480000
40,PARCERIA PARA UTILIZAÇÃO DO AUDITÓRIO,PARCERIA PARA UTILIZAÇÃO DO AUDITÓRIO - A idei...,Relações com a Comunidade,Médio Impacto,4.0,0.6,0.8,3.0,3.0,4.0,5.253333
47,CARRETO SN BRITA,CARRETO SN BRITA - Aquisição de caminhão e mot...,Logística e Cadeia de Suprimentos,Médio Impacto,4.0,0.7,0.8,4.0,3.0,4.0,5.906667
48,Energia para iluminação da Mina,Energia para iluminação da Mina - Atualmente a...,Energia e Eficiência Energética,Médio Impacto,3.0,0.7,0.8,4.0,3.0,4.0,5.346667
77,Impressão das Apostilas de Inglês,Impressão das Apostilas de Inglês - A ideia pa...,Gestão de Recursos Humanos,Baixo Impacto,3.0,0.6,0.8,2.0,2.0,3.0,3.773333
78,Campeonato,Campeonato - Organizar um campeonato interno d...,Gestão de Recursos Humanos,Baixo Impacto,4.0,0.7,0.8,3.0,2.0,4.0,5.240000
79,Quadra de esportes,Quadra de esportes - Construção de uma quadra ...,Relações com a Comunidade,Baixo Impacto,3.0,0.6,0.8,2.0,3.0,2.0,3.773333


# SEM IA PORQUE IA NÃO É TUDO

In [ ]:
from fuzzywuzzy import process
def encontrar_similaridades(texto, df, coluna='IDEIA', limite=30):
    """
    Encontra as linhas no DataFrame que são similares ao texto de entrada usando FuzzyWuzzy.

    Args:
        texto (str): O texto de entrada para comparação.
        df (pd.DataFrame): O DataFrame contendo os dados.
        coluna (str): Nome da coluna no DataFrame a ser comparada.
        limite (int): Limite de similaridade para considerar uma correspondência.

    Returns:
        pd.DataFrame: DataFrame contendo as linhas com similaridade acima do limite.
    """
    # Extrair a coluna de interesse como uma lista
    textos_base = df[coluna].tolist()

    # Encontrar as melhores correspondências
    correspondencias = process.extractBests(texto, textos_base, score_cutoff=limite)

    # Obter os índices das correspondências
    indices = [textos_base.index(m[0]) for m in correspondencias]

    # Retornar o DataFrame com as linhas correspondentes
    return df.iloc[indices]


df_ideias

# Texto de entrada para comparação
texto_input = 'Placas de identificação'

# Encontrar e imprimir as similaridades
resultados = encontrar_similaridades(texto_input, df_ideias)
resultados

,IDEIA_TITULO,IDEIA,categoria,impacto,reach,impact,confidence,business_value,time_criticality,risk_reduction,Valor
3,Pintura e sinalizações na Fábrica,Pintura e sinalizações na Fábrica - Pintura d...,Segurança e Saúde Ocupacional,Baixo Impacto,3.0,0.7,0.8,4.0,3.0,4.0,5.346667
12,Concreto de qualidade é concreto azul,Concreto de qualidade é concreto azul - Pintar...,Construção e Infraestrutura,Baixo Impacto,3.0,0.7,0.8,4.0,3.0,2.0,4.680000
39,DEMONSTRATIVO DE CONTEÚDO MÁQUINA DE CAFÉ,DEMONSTRATIVO DE CONTEÚDO MÁQUINA DE CAFÉ - Co...,Gestão de Recursos Humanos,Ver e Agir,2.0,0.3,0.8,1.0,1.0,1.0,1.480000
50,Plano odontológico,Plano odontológico - Um plano odontológico pa...,Gestão de Recursos Humanos,Baixo Impacto,3.0,0.6,0.7,4.0,3.0,2.0,4.260000
51,organização,organização - deicha um quite de ferramenta em...,Operações de Mineração,Baixo Impacto,3.0,0.5,0.7,2.0,3.0,2.0,3.383333
